In [2]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
from utils import iter_df_read, get_array_diff
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("seaborn")
sns.set(font_scale=1.5)

# Modificar por ruta local de almacenamiento de datos
BASE = "/media/giani/Gianicosas/Magister/Proyecto/"

# Atributos

## Primera iteración
- **Notas de colegio**: Egresados Colegio, `PROM_NOTAS_ALU`.
- **Región del colegio**: Egresados Colegio, `COD_REG_RBD`.
- **Modalidad del colegio (científico humanista / técnico)**: Egresados Colegio, `COD_ENSE`.
- **Tipo de establecimiento (particular, subvencionado, municipal)**: Establecimientos, `cod_depe`.
- **Género**: Matriculados, `gen_alu`.
- **Edad**: Matriculados, `fec_nac_alu`.
- **Área del conocimiento de la carrera**: Matriculados, `area_conocimiento`.
- **Región de la institución de educación superior**: Matriculados, `region_sede`.

## Segunda iteración
- **Valor Arancel**: Matriculados, `valor_arancel` (Matriz 2).
- **Estado acreditación carrera**: Matriculados, `acreditada_carr` (Matriz 2).
- **Estado acreditación institución**: Matriculados, `acreditada_inst` (Matriz 2).
- **Cantidad de becas**: Becas y créditos, `BENEFICIO_BECA_FSCU` (Matriz 2).

## Tercera iteración
- **FSCU**: Tiene fondo solidario, `BENEFICIO_BECA_FSCU` (Matriz 3).
- **gratuidad**: Tiene gratuidad, `BENEFICIO_BECA_FSCU` (Matriz 3).
- **beca**: Tiene beca, `BENEFICIO_BECA_FSCU` (Matriz 3).

## Cuarta iteración
- **Jornada**: Diurno o vespertino (Matriz 4).
- **Duración carrera**: Duración en semestres de la carrera (Matriz 4).
- **Duración titulación**: Duración en semestres del proceso de titulación (Matriz 4).

Se corrige además el valor del arancel, para lo cual se tiene que consultar el formato del valor.

# Lectura datos

## Desertores

In [3]:
desertores_1 = pd.read_csv(f"{BASE}/tmp_data/desertores_1.csv.gz")
desertores_1.head()

,cat_periodo,codigo_unico,mrun,desertor_1
0,2015,I498S6C132J2V1,37.0,0
1,2015,I111S14C309J2V1,118.0,0
2,2015,I106S3C64J4V1,253.0,1
3,2015,I221S1C57J1V1,370.0,1
4,2015,I176S8C59J2V1,533.0,0


## Matrículas

In [4]:
cols_mat = [
    'cat_periodo', 'codigo_unico', 'mrun', 'tipo_inst_1', 
    'gen_alu', 'fec_nac_alu', 'area_conocimiento', 'region_sede',
    "anio_ing_carr_ori",  "valor_arancel", "acreditada_carr", "acreditada_inst",
    "jornada", "dur_estudio_carr", "dur_proceso_tit", "formato_valores"

]

instituciones = ['Institutos Profesionales', 'Centros de Formación Técnica']

matriculados = pd.concat([
    iter_df_read(f, "tipo_inst_1", instituciones, sep=";", usecols=cols_mat)
    for f in tqdm(
        glob(f"{BASE}/raw_data/post_2015/20220719_Matrícula_Ed_Superior_*.csv"),
        total=8
    )
]).dropna(subset="mrun").drop_duplicates().reset_index(drop=True)

matriculados = matriculados[
    (matriculados["anio_ing_carr_ori"] >= 2015)
    & (matriculados["anio_ing_carr_ori"] < 2022)
]

print(f"Cantidad de registros: {matriculados.shape[0]}\n")
print(f"Cantidad de nulos: \n{matriculados.isna().sum()}")
matriculados.head()

  0%|          | 0/8 [00:00<?, ?it/s]

Cantidad de registros: 3202225

Cantidad de nulos: 
cat_periodo                0
codigo_unico               0
mrun                       0
gen_alu                    0
fec_nac_alu                0
anio_ing_carr_ori          0
tipo_inst_1                0
jornada                    0
dur_estudio_carr           0
dur_proceso_tit           17
region_sede                0
formato_valores      2869432
valor_arancel            144
area_conocimiento          0
acreditada_carr            0
acreditada_inst            0
dtype: int64


,cat_periodo,codigo_unico,mrun,gen_alu,fec_nac_alu,anio_ing_carr_ori,tipo_inst_1,jornada,dur_estudio_carr,dur_proceso_tit,region_sede,formato_valores,valor_arancel,area_conocimiento,acreditada_carr,acreditada_inst
0,2015,I374S1C9J1V1,5.0,2,199105,2015,Centros de Formación Técnica,Diurno,5,1.0,Metropolitana,NaN,909000.0,Salud,NO ACREDITADA,ACREDITADA
1,2015,I498S6C132J2V1,37.0,1,197808,2015,Centros de Formación Técnica,Vespertino,5,1.0,Biobío,NaN,820000.0,Tecnología,NO ACREDITADA,ACREDITADA
3,2015,I111S14C309J2V1,118.0,1,199102,2015,Institutos Profesionales,Vespertino,5,1.0,Metropolitana,NaN,1390000.0,Administración y Comercio,ACREDITADA,ACREDITADA
6,2015,I106S3C64J4V1,253.0,1,198404,2015,Institutos Profesionales,A Distancia,5,1.0,Metropolitana,NaN,920000.0,Administración y Comercio,NO ACREDITADA,NO ACREDITADA
8,2015,I221S1C57J1V1,370.0,2,198910,2015,Centros de Formación Técnica,Diurno,5,1.0,Metropolitana,NaN,1880000.0,Administración y Comercio,ACREDITADA,ACREDITADA


In [38]:
matriculados["formato_valores"] = matriculados["formato_valores"].fillna('Monto en Pesos')

In [39]:
mat_des = pd.merge(
    matriculados,
    desertores_1,
    how="left"
)

matriculados.shape, desertores_1.shape, mat_des.shape

((3202225, 16), (2778928, 4), (3202225, 17))

In [40]:
mat_des[mat_des["desertor_1"].isna()]["cat_periodo"].value_counts()

2022    332793
2019     24028
2018     23051
2020     21665
2021     20556
2017     19280
2016     10887
2015      3311
Name: cat_periodo, dtype: int64

In [41]:
mat_des = mat_des.dropna().reset_index(drop=True)

## Colegios

### Egresados EM

In [43]:
cols_egreso_col = [
    'MRUN', 'PROM_NOTAS_ALU', 'COD_REG_RBD', 'RBD', 'COD_ENSE', 'COD_GRADO', "AGNO"
]

egreso_colegio_todos = pd.concat([
    iter_df_read(f, "MRUN", mat_des["mrun"].unique(), sep=";", usecols=cols_egreso_col)
    for f in tqdm(
        glob(f"{BASE}raw_data/egresados_colegios/20220325_Notas_y_Egresados_Enseñanza_Media_*.csv"),
        total=20
    )
]).dropna(subset="MRUN").drop_duplicates().reset_index(drop=True).rename(
   columns={"MRUN": "mrun"}
)

  0%|          | 0/20 [00:00<?, ?it/s]

In [45]:
egreso_colegio_todos["PROM_NOTAS_ALU"] = egreso_colegio_todos["PROM_NOTAS_ALU"].str.replace(",", ".").astype(float)

prom_notas = egreso_colegio_todos.groupby(["mrun"]).agg({
    "PROM_NOTAS_ALU": "mean"
}).reset_index().rename(columns={
    "PROM_NOTAS_ALU": "prom_notas_media"
})

egreso_colegio = pd.merge(
    egreso_colegio_todos,
    prom_notas,
    how="left"
).sort_values(
    by=["COD_GRADO"], ascending=False
).drop_duplicates(
    subset=["mrun"], keep="first"
).reset_index(drop=True)

print(egreso_colegio_todos.shape, egreso_colegio.shape)
print(f"Cantidad de registros: {egreso_colegio.shape[0]}\n")
print(f"Cantidad de nulos: \n{egreso_colegio.isna().sum()}")
egreso_colegio.head()

(3546930, 7) (996543, 8)
Cantidad de registros: 996543

Cantidad de nulos: 
AGNO                0
RBD                 0
COD_REG_RBD         0
COD_ENSE            0
COD_GRADO           0
mrun                0
PROM_NOTAS_ALU      0
prom_notas_media    0
dtype: int64


,AGNO,RBD,COD_REG_RBD,COD_ENSE,COD_GRADO,mrun,PROM_NOTAS_ALU,prom_notas_media
0,2017,10780,13,363,4,25581603,6.3,6.300
1,2015,2375,6,310,4,17321610,6.3,6.375
2,2015,9781,13,610,4,17321154,5.2,4.850
3,2015,40298,4,310,4,17321338,5.1,5.200
4,2009,9985,13,410,4,12817410,4.7,4.525


### Establecimientos
https://datosabiertos.mineduc.cl/directorio-de-establecimientos-educacionales/

- Se modificó los nombres de columnas manualmente en los casos que no coincidían.
- Además, en los archivos antiguos, se agregó la columna para el año.
- Los archivos que venían como xlsx se guardaron de nuevo como xls para facilitar la carga.

In [46]:
establecimientos_04_16 = pd.concat([
    pd.read_csv(f, sep=";").rename(columns={
        "RBD":"rbd",
        "COD_DEPE":"cod_depe",
        "AGNO": "agno",
        "ïagno": "agno"
    })
    for f in tqdm(
        glob(f"{BASE}raw_data/directorio_colegios/Directorio_oficial_*.csv"),
        total=13
    )
])[[
    "rbd", "cod_depe", "agno"
]].dropna()

establecimientos_04_16 = establecimientos_04_16[
    establecimientos_04_16["rbd"].isin(egreso_colegio["RBD"].unique())
].drop_duplicates().reset_index(drop=True)

print(f"Cantidad de registros: {establecimientos_04_16.shape[0]}\n")
print(f"Cantidad de nulos: \n{establecimientos_04_16.isna().sum()}")
establecimientos_04_16.head()

  0%|          | 0/13 [00:00<?, ?it/s]

Cantidad de registros: 48767

Cantidad de nulos: 
rbd         0
cod_depe    0
agno        0
dtype: int64


,rbd,cod_depe,agno
0,1,2,2004
1,3,2,2004
2,4,2,2004
3,5,2,2004
4,7,2,2004


In [47]:
establecimientos_17_21 = pd.concat([
    pd.read_csv(f).rename(columns={
        "RBD":"rbd",
        "COD_DEPE":"cod_depe",
        "AGNO": "agno",
        "ïagno": "agno"
    })
    for f in tqdm(
        glob(f"{BASE}raw_data/directorio_colegios/Directorio_Oficial_*.csv"),
        total=5
    )
])[[
    "rbd", "cod_depe", "agno"
]].dropna()

establecimientos_17_21 = establecimientos_17_21[
    establecimientos_17_21["rbd"].isin(egreso_colegio["RBD"].unique())
].drop_duplicates().reset_index(drop=True)

print(f"Cantidad de registros: {establecimientos_17_21.shape[0]}\n")
print(f"Cantidad de nulos: \n{establecimientos_17_21.isna().sum()}")
establecimientos_17_21.head()

  0%|          | 0/5 [00:00<?, ?it/s]

Cantidad de registros: 21248

Cantidad de nulos: 
rbd         0
cod_depe    0
agno        0
dtype: int64


/tmp/ipykernel_4827/642544561.py:2: DtypeWarning: Columns (6,9,12,14,19,39) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(f).rename(columns={


,rbd,cod_depe,agno
0,1,2,2017
1,3,2,2017
2,4,2,2017
3,5,2,2017
4,7,2,2017


In [48]:
establecimientos_antiguos = pd.concat([
    pd.read_excel(f).rename(columns={
        "RBD":"rbd",
        "COD_DEPE":"cod_depe",
        "ANO": "agno"
    })
    for f in tqdm(
        glob(f"{BASE}raw_data/directorio_colegios/*.xls"),
        total=12
    )
])[[
    "rbd", "cod_depe", "agno"
]].dropna()

establecimientos_antiguos = establecimientos_antiguos[
    establecimientos_antiguos["rbd"].isin(egreso_colegio["RBD"].unique())
].drop_duplicates().reset_index(drop=True)

print(f"Cantidad de registros: {establecimientos_antiguos.shape[0]}\n")
print(f"Cantidad de nulos: \n{establecimientos_antiguos.isna().sum()}")
establecimientos_antiguos.head()

  0%|          | 0/12 [00:00<?, ?it/s]

Cantidad de registros: 33573

Cantidad de nulos: 
rbd         0
cod_depe    0
agno        0
dtype: int64


,rbd,cod_depe,agno
0,1,2,1992
1,3,2,1992
2,4,2,1992
3,5,2,1992
4,7,2,1992


In [49]:
establecimientos_antiguos.isna().sum()

rbd         0
cod_depe    0
agno        0
dtype: int64

In [50]:
establecimientos = pd.concat([establecimientos_04_16, establecimientos_17_21, establecimientos_antiguos])
establecimientos.head()

,rbd,cod_depe,agno
0,1,2,2004
1,3,2,2004
2,4,2,2004
3,5,2,2004
4,7,2,2004


In [51]:
sorted(establecimientos["agno"].unique())

[1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021]

In [52]:
colegios = pd.merge(
    egreso_colegio.rename(columns={
        "RBD":"rbd",
        "AGNO": "agno"
    }),
    establecimientos,
    how="left"
).drop_duplicates().reset_index(drop=True)

colegios.shape, egreso_colegio.shape

((996543, 9), (996543, 8))

In [53]:
colegios.head()

,agno,rbd,COD_REG_RBD,COD_ENSE,COD_GRADO,mrun,PROM_NOTAS_ALU,prom_notas_media,cod_depe
0,2017,10780,13,363,4,25581603,6.3,6.300,1
1,2015,2375,6,310,4,17321610,6.3,6.375,2
2,2015,9781,13,610,4,17321154,5.2,4.850,3
3,2015,40298,4,310,4,17321338,5.1,5.200,3
4,2009,9985,13,410,4,12817410,4.7,4.525,1


In [54]:
colegios[colegios["cod_depe"].isna()].shape

(3659, 9)

### Becas
https://datosabiertos.mineduc.cl/asignaciones-de-becas-y-creditos-en-educacion-superior/

In [55]:
becas = pd.concat([
    pd.read_csv(f, sep=";")
    for f in tqdm(
        glob(f"{BASE}raw_data/becas_post_2015/Asignacion*.csv"),
        total=6
    )
]).drop(columns=[
    "TIPO_ALUMNO"
]).rename(columns={
    "ANIO_BENEFICIO": "cat_periodo",
    "MRUN": "mrun"
})

becas = becas[
    becas["mrun"].isin(matriculados["mrun"].unique())
].drop_duplicates().reset_index(drop=True)

print(f"Cantidad de registros: {becas.shape[0]}\n")
print(f"Cantidad de nulos: \n{becas.isna().sum()}")
becas.head()

  0%|          | 0/6 [00:00<?, ?it/s]

Cantidad de registros: 1712062

Cantidad de nulos: 
cat_periodo            0
mrun                   0
BENEFICIO_BECA_FSCU    0
dtype: int64


,cat_periodo,mrun,BENEFICIO_BECA_FSCU
0,2015,370,BNM
1,2015,598,BNM
2,2015,1118,BNM
3,2015,1177,BNM
4,2015,1434,BNM


In [56]:
# Tercera iteración
becas["fscu"] = np.where(becas["BENEFICIO_BECA_FSCU"] == "FSCU", 1, 0)
becas["gratuidad"] = np.where(becas["BENEFICIO_BECA_FSCU"] == "GRATUIDAD", 1, 0)
becas["beca"] = np.where(~becas["BENEFICIO_BECA_FSCU"].isin(["FSCU", "GRATUIDAD"]), 1, 0)

# 1712062
print(becas["fscu"].value_counts())
print(becas["gratuidad"].value_counts())
print(becas["beca"].value_counts())

0    1701930
1      10132
Name: fscu, dtype: int64
0    893392
1    818670
Name: gratuidad, dtype: int64
1    883260
0    828802
Name: beca, dtype: int64


In [57]:
becas_merge = becas.drop(columns=["BENEFICIO_BECA_FSCU"]).drop_duplicates().groupby(["cat_periodo", "mrun"]).agg({
    "fscu": "sum",
    "gratuidad": "sum",
    "beca": "sum"
}).reset_index()
print(becas_merge.shape, becas.shape)

(1695636, 5) (1712062, 6)


In [58]:
becas_merge["fscu"].max(), becas_merge["gratuidad"].max(), becas_merge["beca"].max(),

(1, 1, 1)

## Merge matriz 1

In [59]:
matriz_1 = pd.merge(
    mat_des,
    colegios[["mrun", "COD_REG_RBD", "COD_ENSE", "prom_notas_media", "cod_depe"]].rename(columns={
        "COD_REG_RBD":"region_colegio",
        "COD_ENSE": "tipo_ensenanza_colegio", #CH, tecnico, etc
        "cod_depe": "dependencia_colegio" #municipal, subvencionado, etc
    }),
    how="left"
).drop_duplicates().reset_index(drop=True)

colegios.shape, mat_des.shape, matriz_1.shape

((996543, 9), (2746497, 17), (2746497, 21))

In [60]:
matriz_1.isna().sum()

cat_periodo                    0
codigo_unico                   0
mrun                           0
gen_alu                        0
fec_nac_alu                    0
anio_ing_carr_ori              0
tipo_inst_1                    0
jornada                        0
dur_estudio_carr               0
dur_proceso_tit                0
region_sede                    0
formato_valores                0
valor_arancel                  0
area_conocimiento              0
acreditada_carr                0
acreditada_inst                0
desertor_1                     0
region_colegio            349041
tipo_ensenanza_colegio    349041
prom_notas_media          349041
dependencia_colegio       356773
dtype: int64

## Merge Matriz 2

## Merge Matriz 3

In [61]:
matriz_3 = pd.merge(
    matriz_1,
    becas_merge,
    how="left"
).drop_duplicates().reset_index(drop=True)

for c in ["fscu", "gratuidad", "beca"]:
    matriz_3[c] = matriz_3[c].fillna(0)

print(becas_merge.shape, matriz_1.shape, matriz_3.shape)

(1695636, 5) (2746497, 21) (2746497, 24)


In [62]:
matriz_3.isna().sum()

cat_periodo                    0
codigo_unico                   0
mrun                           0
gen_alu                        0
fec_nac_alu                    0
anio_ing_carr_ori              0
tipo_inst_1                    0
jornada                        0
dur_estudio_carr               0
dur_proceso_tit                0
region_sede                    0
formato_valores                0
valor_arancel                  0
area_conocimiento              0
acreditada_carr                0
acreditada_inst                0
desertor_1                     0
region_colegio            349041
tipo_ensenanza_colegio    349041
prom_notas_media          349041
dependencia_colegio       356773
fscu                           0
gratuidad                      0
beca                           0
dtype: int64

In [63]:
matriz_3.to_csv(f"{BASE}work_data/matriz_desercion_4.csv.gz", index=False, compression="gzip")